# Critical Temperature of Superconductors

- In order to compare in detail the results of different hyperparameters configurations, it is developed a system based on GridSearchcv[*](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and Pipeline[*](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) that execute a single configuration at each execution, and save it into a csv file. A different csv file is generated for each model.
    - Another advantage of this system is that the program execution can be stopped at any time without losing the already trained configurations
    - The only downside is that the execution is not parallel, but the dataset is relateively small, thus not much time for each configuration execution
- 

In [43]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor

from sklearn import preprocessing

from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import GridSearchCV

import os
from sklearn.metrics import r2_score, mean_squared_error

from utils import Step, Pipe, Combination, extract_combinations, combination_already_tested, print_results

In [44]:
plt.style.use("seaborn-v0_8")

DATA_FOLDER = "data/"
OUTPUT_FOLDER = "outputs/"

In [45]:
RANDOM_STATE = 42

OUTLIER_REMOVAL = False

---
---
## Data Load

In [46]:
df = pd.concat(
    [
        pd.read_csv(DATA_FOLDER + "formula_train.csv").drop(columns=["critical_temp"]),
        pd.read_csv(DATA_FOLDER + "train.csv"),
    ],
    axis=1,
)
print("Shapes of Properties+Formula df: ", df.shape)

Shapes of Properties+Formula df:  (17010, 169)


In [47]:
# Remove "material" feature
df = df.drop(columns="material")

---
---
## Split

In [48]:
train, test = train_test_split(df, test_size=0.2, random_state=RANDOM_STATE)

X_train = train.drop(columns=["critical_temp"])
y_train = train[["critical_temp"]]

X_test = test.drop(columns=["critical_temp"])
y_test = test[["critical_temp"]]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13608, 167), (3402, 167), (13608, 1), (3402, 1))

---
---
## Remove Highly correlated features

In [49]:
# TODO: remove, useless
# if REMOVE_HIGH_CORR_FEATURES:
#     corr_matrix = df.corr().abs()
#     upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
#     cols_to_drop = [column for column in upper.columns if any(upper[column] >= CORR_THRESHOLD)]

#     print("{} Cols Removed: {}".format(len(cols_to_drop), cols_to_drop))
#     X_train = X_train.drop(columns=cols_to_drop)
#     X_test = X_test.drop(columns=cols_to_drop)
#     print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [50]:
class FeaturesRemover:
    """
    Class that provide the fit and transform methods, in order to be used as a "transformer" into the Pipeline class
    """

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        corr_matrix = df.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        cols_to_drop = [column for column in upper.columns if any(upper[column] >= self.corr_threshold)]

        # print("{} Cols Removed: {}".format(len(cols_to_drop), cols_to_drop))
        X = X.drop(columns=cols_to_drop)
        return X

    def set_params(self, corr_threshold):
        self.corr_threshold = corr_threshold
        return self

---
---
## Outlier removal

In [51]:
if OUTLIER_REMOVAL:
    columns = train.columns
    outliers = pd.Series(index=train.index, dtype=bool)

    clf = LocalOutlierFactor(n_jobs=-1)
    # clf = IsolationForest(
    #     max_samples=1.0,
    #     contamination=0.001,
    #     n_jobs=-1,
    #     random_state=random_state,
    # )
    outliers = clf.fit_predict(train) == -1

    print("Outliers removed: {}".format(outliers.sum()))
    train = train[~outliers]

    X_train = train.drop(columns=["critical_temp"])
    y_train = train[["critical_temp"]]
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [52]:
# TODO: Remove, wrong bc not applicable
# class OutliersRemover:
#     def __init__(self) -> None:
#         self.outliers_vector = None

#     def fit(self, X, y):
#         self.outliers_vector = pd.Series(index=X.index, dtype=bool)

#         clf = LocalOutlierFactor(n_jobs=-1)
#         # clf = IsolationForest(
#         #     max_samples=1.0,
#         #     contamination=0.001,
#         #     n_jobs=-1,
#         #     random_state=random_state,
#         # )
#         self.outliers_vector = clf.fit_predict(np.column_stack((X, y))) == -1
#         print("Outliers removed: {}".format(self.outliers_vector.sum()))
#         return self

#     def transform(self, X, y=None):

#         X = X[~self.outliers_vector]
#         # y = y[~self.outliers_vector]

#         print("Outliers removed: {}".format(self.outliers_vector.sum()))
#         print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
#         return X  # , y

#     def set_params(self):
#         return self

---
---
## Preprocessing

In [53]:
features_remover_step = Step(
    "features_remover",
    FeaturesRemover(),
    {"corr_threshold": 0.95},
)
std_step = Step(
    "std",
    preprocessing.StandardScaler(),
)
minmax_step = Step(
    "minmax",
    preprocessing.MinMaxScaler(),
)
l1_step = Step(
    "l1",
    preprocessing.Normalizer(norm="l1"),
)
l2_step = Step(
    "l2",
    preprocessing.Normalizer(norm="l2"),
)
lmax_step = Step(
    "lmax",
    preprocessing.Normalizer(norm="max"),
)
pca_step = Step(
    "pca",
    PCA(random_state=RANDOM_STATE),
    {
        "n_components": [0.95],
        # "whiten": [True, False],
        # "svd_solver": "full",
    },
)

---
---
## Search

In [54]:
def grid_search(combinations: list[Combination], estimator_tag: str, save_results=True):

    # Iterate over *all* combinations
    for index, combination in enumerate(combinations):
        print("\nCombination {}/{}  |  {}".format(index + 1, len(combinations), combination.tag))

        # Check if this combination is already tested
        if save_results:
            file_name = OUTPUT_FOLDER + estimator_tag + "_output.csv"
            if combination_already_tested(file_name, combination):
                print("  ==> Already done. Skipped.")
                continue

        gs = GridSearchCV(
            estimator=combination.pipeline,
            param_grid=combination.parameters,
            n_jobs=-1,
            cv=3,
            verbose=0,
        )

        # Fit
        gs.fit(X_train, np.ravel(y_train))
        # Predict
        y_pred = gs.predict(X_test)
        # Test scores
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print("  ==> R2: {}\tMSE: {}".format(r2, mse))

        # Save results
        if save_results:
            results = combination.set_MSE(mse).set_R2(r2).as_df()
            if os.path.isfile(file_name):
                outputs = pd.read_csv(file_name)
                if not outputs.empty:
                    results = pd.concat([outputs, results], axis=0)
            results.to_csv(file_name, index=False)

In [55]:
estimator_tag = "random_forest"
random_forest_step = Step(
    estimator_tag,
    RandomForestRegressor(n_jobs=-1, random_state=RANDOM_STATE),
    {
        "max_samples": [0.66],
        "criterion": ["squared_error"],
        "n_estimators": [200],
        "max_depth": [25],  # , 5, 10, 15, 20],
        # "max_leaf_nodes": [None],  # 50, 100, 200, 300, 400
        # "ccp_alpha": [0.0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0],
        "max_features": [0.4],  # "sqrt", "log2", 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.9
    },
)

combinations = extract_combinations(
    Pipe(random_forest_step),
    Pipe(minmax_step, random_forest_step),
    Pipe(minmax_step, pca_step, random_forest_step),
    Pipe(features_remover_step, minmax_step, pca_step, random_forest_step),
    Pipe(std_step, random_forest_step),
    Pipe(std_step, pca_step, random_forest_step),
    Pipe(features_remover_step, std_step, pca_step, random_forest_step),
    Pipe(minmax_step, l1_step, random_forest_step),
    Pipe(features_remover_step, minmax_step, l1_step, random_forest_step),
    Pipe(lmax_step, random_forest_step),
)
grid_search(combinations, estimator_tag=estimator_tag)


Combination 1/10  |  random_forest
  ==> Already done. Skipped.

Combination 2/10  |  minmax + random_forest
  ==> Already done. Skipped.

Combination 3/10  |  minmax + pca + random_forest
  ==> Already done. Skipped.

Combination 4/10  |  features_remover + minmax + pca + random_forest
  ==> Already done. Skipped.

Combination 5/10  |  std + random_forest
  ==> Already done. Skipped.

Combination 6/10  |  std + pca + random_forest
  ==> Already done. Skipped.

Combination 7/10  |  features_remover + std + pca + random_forest
  ==> Already done. Skipped.

Combination 8/10  |  minmax + l1 + random_forest
  ==> Already done. Skipped.

Combination 9/10  |  features_remover + minmax + l1 + random_forest
  ==> Already done. Skipped.

Combination 10/10  |  lmax + random_forest
  ==> Already done. Skipped.


In [56]:
print_results(OUTPUT_FOLDER + estimator_tag + "_output.csv", 5)

,tag,R2,MSE,random_forest__max_samples,random_forest__criterion,random_forest__n_estimators,random_forest__max_depth,random_forest__max_features,pca__n_components,features_remover__corr_threshold,random_forest__max_leaf_nodes
4,std + random_forest,0.9269,83.2301,0.6600,squared_error,200,25,0.4000,nan,nan,nan
14,std + random_forest,0.9269,83.2301,0.6600,squared_error,200,25,0.4000,nan,nan,'None'
11,minmax + random_forest,0.9268,83.2731,0.6600,squared_error,200,25,0.4000,nan,nan,'None'
1,minmax + random_forest,0.9268,83.2731,0.6600,squared_error,200,25,0.4000,nan,nan,nan
21,lmax + random_forest,0.9267,83.3483,0.6600,squared_error,200,25,0.4000,nan,nan,nan
